In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from os import chdir

In [2]:
#планируется словарь алгоритмов для более быстрого создания всех датафреймов
alg_dict = {'sort': 'bubble_sort','find': 'binary_search'}

In [3]:
#создание датафрейма
def make_df(filename):
    df = pd.read_csv(f'csv/{filename}.csv')
    df = df.set_index('filename')
    del df['Unnamed: 0']
    return df

In [4]:
#чтение и преобразование файла
def file_read(data_alg,df):
    chdir('dataset/'+data_alg)
    new_lines = []
    for index, row in df.iterrows():
        with open(str(index),'r', encoding = 'utf-8') as f:
            lines = f.readlines()
            lines = [line for line in lines if line.strip() != ""]
            #lines = lines
        new_lines.append(lines)
    %cd ../..
    return new_lines

In [5]:
#обработка переменной без табов, заменяя их на 2 пробела (Доработать)
def preporcessing_file(lines):
    new_lines = []
    code_line = ''
    for i in lines:
        for j in i:
            j = j.replace('    ',' ')
            code_line +=j+''
            new_lines.append(code_line)
        code_line = ''
    return new_lines

In [10]:
df_bubble_sort = make_df('buble_sort')
file = file_read('dataset_bubble_sort',df_bubble_sort)

C:\Users\Matvey\Анализ кода\Code-Insight


In [12]:
file

[['def bubble_sort(list1):\n',
  '    has_swapped = True\n',
  '    total_iteration = 0\n',
  '    while (has_swapped):\n',
  '        has_swapped = False\n',
  '        for i in range(len(list1) - total_iteration - 1):\n',
  '            if list1[i] > list1[i + 1]:\n',
  '                # Swap\n',
  '                list1[i], list1[i + 1] = list1[i + 1], list1[i]\n',
  '                has_swapped = True\n',
  '        total_iteration += 1\n',
  '    print("The number of iteraton: ", total_iteration)\n',
  '    return list1\n'],
 ['def bubble_sort(list1):\n',
  '    # We can stop the iteration once the swap has done\n',
  '    has_swapped = True\n',
  '    while (has_swapped):\n',
  '        has_swapped = False\n',
  '        for i in range(len(list1) - 1):\n',
  '            if list1[i] > list1[i + 1]:\n',
  '                # Swap\n',
  '                list1[i], list1[i + 1] = list1[i + 1], list1[i]\n',
  '                has_swapped = True\n',
  '    return list1\n'],
 ['def bubb

# Вспомогательные функции

In [67]:
pwd

'C:\\Users\\Matvey\\Анализ кода\\Code-Insight\\dataset\\dataset_bubble_sort'

In [22]:
%cd Анализ кода/Code-Insight

C:\Users\Matvey\Анализ кода\Code-Insight


In [82]:
%cd ../..

C:\Users\Matvey\Анализ кода\Code-Insight
